## nfl lines machine learning project


source data for historical spreads and game scores
http://www.repole.com/sun4cast/data.html

In [3]:
import os

# define the root directory for the nfl code in $MLNLF_ROOT

#codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
codeDir = "".join(['/Users/amit/repos/mlnfl/nfl', os.path.sep])
dataRoot = "".join([codeDir, "data", os.path.sep])

os.chdir(codeDir)

print(codeDir)
print(dataRoot)

/Users/amit/repos/mlnfl/nfl/
/Users/amit/repos/mlnfl/nfl/data/


In [4]:
# warnings control
import warnings
# choose default, ignore, always
warnings.filterwarnings('default')


In [5]:
# import necessary modules
%matplotlib inline

from __future__ import division
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import madden

from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble

print ("pandas version ",pd.__version__)

pandas version  0.16.2


In [6]:
# location of lookup files
from referencedata import ReferenceData
lookupFiles = { 'teams' : {'file': 'nflTeams.csv' },
                'seasons' : {'file': 'seasons.csv' },
               }

lookupDir = "".join([dataRoot, 'lookup', os.path.sep])

print ("lookupFiles = %s" % lookupFiles)
print ("lookupDir = %s" % lookupDir)

lookupFiles = {'seasons': {'file': 'seasons.csv'}, 'teams': {'file': 'nflTeams.csv'}}
lookupDir = /Users/amit/repos/mlnfl/nfl/data/lookup/


In [7]:
# import reference data
reference_data = ReferenceData(lookupDir)
reference_data.teams_df.head()

,city,mascot,league,division,year
team,,,,,
Baltimore Ravens,NaN,Ravens,afc,north,NaN
New England Patriots,NaN,Patriots,afc,east,NaN
Tennessee Titans,NaN,Titans,afc,south,NaN
Atlanta Falcons,NaN,Falcons,nfc,south,NaN
Tampa Bay Buccaneers,NaN,Buccaneers,nfc,south,NaN


In [8]:
###  multi-season training
reload(madden)

# testYears and trainYears need to be arrays
trainYears = [2014] #range(2008,2013) 
testYear = [2015]

# train on previous 3 yrs of data
testYear = [2015]
trainYears = range(testYear[0]-3,testYear[0]) 

# training data set - includes one extra year for prev yr record
seasons = np.array(trainYears)
print ("training seasons >> ", seasons)

training seasons >>  [2012 2013 2014]


In [10]:
reload(madden)

# get training data
# 1 - read all the games
path_to_lines = dataRoot + "lines/"
dfAllGames = madden.readGamesAll(path_to_lines, seasons)
# 2 - compile season record for all teams
dfAllTeams = madden.seasonRecord(dfAllGames, reference_data)
# 3 - apply season records and compute other fields for all games
dfAllGames = madden.processGames(dfAllGames, dfAllTeams, reference_data)
# 4 - remove extra year of data 
dfAllGames = dfAllGames[dfAllGames.season.isin(seasons)]

# use different test set 
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)
# 1 - read all the games
dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)  
# 2 - compile season record for all teams
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data) 
# 3 - apply season records and compute other fields for all games
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data) 
# 4 - remove extra year of data 
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]


madden.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['gamesPlayed'] = range(1, len(team_df.index) + 1)  # index 1 thur 16
madden.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['team'] = team
madden.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['homeGame'] = se

results for >>  [2015]


/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
# define independent variables for logistic regression
features = ['favoredRecord','underdogRecord',  # current year records of both teams
            'prevFavoredRecord','prevUnderdogRecord', # prev year records, helps early in season when only few games played
            'gameWeek',  # week in season, should make a good/bad record later in season more important
            'absLine',  # absolute value of spread since favored team already determined
            'divisionGame', # T/F, usually more competitive rivalry games, i.e. bad teams still win home division games.
            'favoredHomeGame', # T/F, important since output of classifier is "did the favored team win?"
            ]

# run the classifer
random_state = 11
#classifier = svm.SVC(kernel='poly',probability=True, random_state=random_state)
classifier = linear_model.LogisticRegression(C=1e5)

mlClassifier = madden.runScikitClassifier(dfAllGames,features,classifier)

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfGamesTest,mlClassifier,features)

# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict,reference_data,seasonTest[0])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [14]:
# diagnostics ... weekly outcomes
g = dfAll.groupby('gameWeek')['lineScore','probaScore1','probaScore2','probaScore3'].sum()
print(g)

g.sum()

NameError: name 'dfAll' is not defined

In [13]:
# predict one week of current season
iweek = 7
reload(madden)

# use different test set - current year
testYear = [2015]
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)

dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]

# pick only this weeks games for predict
dfTest = dfGamesTest[dfGamesTest.gameWeek == iweek]

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfTest,mlClassifier,features)
# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict, reference_data, seasonTest[0])



results for >>  [2015]


madden.py:411: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfPredict['predict_proba'] = pp[:, 1]
madden.py:412: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfPredict['predict_proba_abs'] = abs(pp[:, 1] - 0.5)
madden.py:418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfPredict['predictWin'] = 1 * ((s

In [22]:
print (dataRoot)
dfAllGames.to_csv(dataRoot + '/allgames.csv', index= False)

/Users/amit/repos/mlnfl/nfl/data/


In [14]:
# display weekly ranking output

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

dispCols = ['season','gameWeek','Visitor','visitorRecord','Home Team','homeRecord',
            'Line','prevFavoredRecord','prevUnderdogRecord','predict_proba',
            'lineGuess','probaGuess', 'probaAbsGuess', 'predictTeam']


dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
guessCol = 'probaGuess'
predictCols = ['gameWeek','predictTeam', 'predict_proba', guessCol, 'favorite','lineGuess', 'Line']
dfAll[predictCols].sort(guessCol, ascending=False)

,gameWeek,predictTeam,predict_proba,probaGuess,favorite,lineGuess,Line
8786,7,New England Patriots,0.864424,16,New England Patriots,16,9.0
8790,7,Arizona Cardinals,0.829496,15,Arizona Cardinals,15,7.5
8789,7,Carolina Panthers,0.711956,14,Carolina Panthers,5,3.0
8788,7,New York Giants,0.693856,13,New York Giants,7,3.5
8780,7,Atlanta Falcons,0.689779,12,Atlanta Falcons,9,-4.5
8784,7,St Louis Rams,0.685367,11,St Louis Rams,13,5.5
8782,7,Minnesota Vikings,0.681642,10,Minnesota Vikings,4,-2.5
8777,7,Seattle Seahawks,0.675882,9,Seattle Seahawks,14,-6.0
8785,7,Miami Dolphins,0.662626,8,Miami Dolphins,11,4.5
8781,7,Indianapolis Colts,0.651330,7,Indianapolis Colts,10,4.5


In [15]:
week_filter = dfAll.gameWeek == 7
dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
dfAll[week_filter][predictCols].sort(guessCol, ascending=False)

,gameWeek,predictTeam,predict_proba,probaGuess,favorite,lineGuess,Line
8786,7,New England Patriots,0.864424,16,New England Patriots,16,9.0
8790,7,Arizona Cardinals,0.829496,15,Arizona Cardinals,15,7.5
8789,7,Carolina Panthers,0.711956,14,Carolina Panthers,5,3.0
8788,7,New York Giants,0.693856,13,New York Giants,7,3.5
8780,7,Atlanta Falcons,0.689779,12,Atlanta Falcons,9,-4.5
8784,7,St Louis Rams,0.685367,11,St Louis Rams,13,5.5
8782,7,Minnesota Vikings,0.681642,10,Minnesota Vikings,4,-2.5
8777,7,Seattle Seahawks,0.675882,9,Seattle Seahawks,14,-6.0
8785,7,Miami Dolphins,0.662626,8,Miami Dolphins,11,4.5
8781,7,Indianapolis Colts,0.651330,7,Indianapolis Colts,10,4.5


In [18]:
# display weekly ranking output for spread method

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

predictCols = ['favorite','lineGuess', 'absLine','Line', 'favoredHomeGame', 'divisionGame', 'favoredRecord']

sortCols = ['absLine','favoredHomeGame', 'divisionGame', 'favoredRecord', 'favorite']
dfSpread = dfAll[predictCols].sort(sortCols , ascending=False)
#print(dfSpread.to_csv(sys.stdout,sep=',', index=False))
dfSpread

,favorite,lineGuess,absLine,Line,favoredHomeGame,divisionGame,favoredRecord
8786,New England Patriots,16,9.0,9.0,1,1,0.833333
8790,Arizona Cardinals,15,7.5,7.5,1,0,0.666667
8777,Seattle Seahawks,14,6.0,-6.0,0,1,0.333333
8784,St Louis Rams,13,5.5,5.5,1,0,0.333333
8778,Buffalo Bills,12,5.5,-5.5,0,0,0.500000
8781,Indianapolis Colts,11,4.5,4.5,1,0,0.500000
8785,Miami Dolphins,10,4.5,4.5,1,0,0.333333
8780,Atlanta Falcons,9,4.5,-4.5,0,0,0.833333
8787,San Diego Chargers,8,4.0,4.0,1,1,0.333333
8788,New York Giants,7,3.5,3.5,1,1,0.500000


In [30]:
# loop over multiple years

import runMadden
import walsh
reload(madden)
reload(runMadden)


trainStart = 2010
trainLen = 3

random_state = None
mlClassifier = svm.SVC(kernel='poly',probability=True, random_state=random_state)
dfSVM = runMadden.runSeasonLoop(trainStart, trainLen, mlClassifier, path_to_lines, reference_data)

mlClassifier = linear_model.LogisticRegression(C=1e5)
dfLog = runMadden.runSeasonLoop(trainStart, trainLen, mlClassifier, path_to_lines, reference_data)
#dfLoop = dfSVM


2013 [2010, 2011, 2012]
1653 <type 'int'>
2014 [2011, 2012, 2013]
1664 <type 'int'>
2013 [2010, 2011, 2012]
1653 <type 'int'>
2014 [2011, 2012, 2013]
1664 <type 'int'>


In [31]:
dfSVM

,lineScore,probaScore1,probaScore2,probaScore3,trainYears,classifierType,classifier
season,,,,,,,
2013,2,8,2,-17,[2010 2011 2012],<class 'sklearn.svm.classes.SVC'>,"SVC(C=1.0, cache_size=200, class_weight=None, ..."
2014,5,85,99,113,[2011 2012 2013],<class 'sklearn.svm.classes.SVC'>,"SVC(C=1.0, cache_size=200, class_weight=None, ..."


In [32]:
dfLog

,lineScore,probaScore1,probaScore2,probaScore3,trainYears,classifierType,classifier
season,,,,,,,
2013,2,28,40,35,[2010 2011 2012],<class 'sklearn.linear_model.logistic.Logistic...,"LogisticRegression(C=100000.0, class_weight=No..."
2014,5,5,-35,-47,[2011 2012 2013],<class 'sklearn.linear_model.logistic.Logistic...,"LogisticRegression(C=100000.0, class_weight=No..."
